In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup

from selenium.common.exceptions import TimeoutException
import pymongo
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
# 创建MongoClient类对象
client = pymongo.MongoClient(host="localhost", port=27017)
mongo = client.digits_ecom3
collections = mongo.papers3

# 定义搜索函数，根据传入的关键字搜索
def searcher(try_1):
    driver.get('https://kns.cnki.net/kns8/AdvSearch?dbcode=CFLQ')
    time.sleep(2)

    # 通过id属性定位"搜索"编辑框节点，并赋值给input
    wait.until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, '#gradetxt > dd:nth-child(2) > div.input-box > input[type=text]')
        )
    ).send_keys(try_1)
    time.sleep(2)



    wait.until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, '#JournalSourceType > label:nth-child(5) > input[type=checkbox]')
        )
    ).click()
    time.sleep(2)


    wait.until(
        EC.presence_of_element_located(
            (By.XPATH,
             '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div/input')
        )
    ).send_keys(2017)
    time.sleep(2)


    wait.until(
        EC.presence_of_element_located(
            (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/input')
        )
    ).click()
    time.sleep(3)
    
    wait.until(EC.presence_of_element_located(
    (By.XPATH, '//*[@id="orderList"]/li[2]')
    )
    ).click()
    time.sleep(3)
    
    wait.until(EC.presence_of_element_located(
    (By.XPATH, '//*[@id="orderList"]/li[2]')
    )
    ).click()
    time.sleep(3)

    # 定位每页文章篇数列表节点并单击
    wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, '[class = "icon icon-sort"]')
    )
    ).click()
    time.sleep(3)
    
    # 定位每页文章篇数"50"节点并单击
    wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, ' #id_grid_display_num ul li')
    )
    )[2].click()
    
    time.sleep(3)

    parse_page_num() #调用parse_page()函数


# 定义解析网页函数
def parse_page_num():
    wait.until(
        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, '.result-table-list tbody tr')
        )
    )
    html = driver.page_source  # 获取HTML
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select('.result-table-list tbody tr')
        # 遍历列表，提取文章信息
    for i in range(0, len(items)):
        # 定位文章链接
        wait.until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="gridTable"]/table/tbody/tr[{}]/td[2]/a'.format(i+1))
    )
    ).click()
        time.sleep(1)
        
        item = items[i]
        detail = item.select('td')
            
        # 跳转页面
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)
        
       
        institution = driver.find_element(By.CSS_SELECTOR,'body > div.wrapper > div.main > div.container > div > div > div.brief > div > h3:nth-child(3)').text 
      
        abstract = driver.find_element(By.XPATH, '//*[@id="ChDivSummary"]').text
        key_words = driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div[3]/div/div/div[5]').text 
        
        
        
        paper = {
            'order':i,
            'seq':detail[0].text.strip(),
            'name':detail[1].text.strip(),
            'institution':institution,
            'abstract':abstract,
            'key_words':key_words,      
            'source':detail[3].text.strip(),
            'date': detail[4].text.strip()
        }
        print(paper)
        data_storage(paper)  # 调用data_storage函数
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        

def data_storage(paper):
    try:
        collections.insert_one(paper)
    except Exception:
        print('failedly storage!', paper)
        

# 定义翻页函数
def next_page():
    try:
        # 判断id值为"Page_next_top"的节点是否可见
        page_next = wait.until(
            EC.visibility_of_element_located(
                (By.CSS_SELECTOR, '#Page_next_top')
            )
        )
    except TimeoutException:  # 捕获超时异常
        return False
    else:
        page_next.click()
        time.sleep(1)
        return True


if __name__ == '__main__':
    try_1 = '数字经济'
    searcher(try_1)
    while True:
        flag = next_page()
        time.sleep(5)
        if flag:
            parse_page_num()
            
            continue
        else:
            break
    driver.close()

{'order': 0, 'seq': '1', 'name': '近年来国内外面向创新创业的信息资源保障政策分析', 'institution': '1. 武汉大学信息管理学院2. 苏州大学社会学院', 'abstract': '【目的/意义】全球范围内新兴信息科技领域涌现的创新创业潮越来越呈现出各领域开放、协同的网状特征。我国也应积极构建相关政策框架,为未来科技创新和经济转型铺路。【方法/过程】采用文献调研和对比分析方法,从微观、中观、宏观三个层面梳理了近年来主要发达国家和地区面向创新创业的信息资源保障政策,认为我国也可以借鉴这三个层面的经验,从面向中小初创企业的金融支持政策、保障科学和知识创新的科研政策、改革经济发展方式的数字经济战略入手构建非技术配套机制。【结果/结论】适应未来经济和社会积木式、开放式的创新体系发展趋势。 ', 'key_words': '关键词： 创新创业; 信息资源保障; 政策分析; 创新体系;', 'source': '情报科学', 'date': '2017-03-05'}
{'order': 1, 'seq': '2', 'name': '中国分享经济发展现状、问题及趋势', 'institution': '  分享经济研究中心    国家信息中心  ', 'abstract': '对2016年中国分享经济发展呈现出的现状特点、问题以及发展方向进行了全面分析,并概括总结了交通出行、房屋住宿、知识技能、生活服务、医疗服务、生产能力等六个重点行业的分享经济发展情况。2016年中国分享经济快速发展,在培育经济发展新动能、引领创新、带动就业等方面发挥了举足轻重的作用,同时经济竞争格局、行业热点、政策环境和平台治理也发生了重大变化。分享经济在快速发展过程中也遭遇到各种问题,主要集中在政策法规滞后、诚信体系不健全、公共数据获取难和统计监测体系亟待建立等方面。未来几年,分享经济仍将继续保持高速增长,并将加速与实体经济融合,重塑就业形态,大型平台企业也将加速全球化布局和生态化扩张,而分享经济治理必将走向包容性监管与多元化协同。 ', 'key_words': '关键词： 分享经济; 共享经济; 数字经济; 平台治理; 网络社会;', 'source': '电子政务', 'date': '2017-03-20'}
{'order': 2, 'seq'